## This file contains experiments done for results section - for calculating scores for table 7


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import pandas as pd
train_df = pd.read_csv('submission_train.csv')
train_df.head()

,line,label,emoji_scores,fake_lables,spam_score,vadar,emotion_scores,sentiment_scores
0,<user> i dunno justin read my mention or not ....,1,0.0,0,0,0.6597,1,1
1,"because your logic is so dumb , i will not eve...",1,0.0,0,0,-0.6418,-1,-1
2,<user> just put casper in a box ! looved the b...,1,0.0,0,0,-0.7896,1,1
3,<user> <user> thanks sir > > do not trip lil m...,1,0.0,0,0,0.4926,1,1
4,visiting my brother tmr is the bestest birthda...,1,0.0,0,0,0.5826,1,1


In [ ]:
print(len(train_df))

2270482


## Swap orders cause there were differences in ways my team mate and i built the training dataframe

In [ ]:
samuels_pred_df = pd.read_csv('training_no_duplicates.csv')

In [ ]:
train_df.loc[:, 'sentiment_scores'] = train_df.loc[:, 'sentiment_scores'].replace(0,-1)
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].replace(0, np.nan)
train_df.loc[:,'spam_score'] = train_df['spam_score'].replace(0,np.nan)
train_df.loc[:,'fake_lables'] = train_df['fake_lables'].replace(0,np.nan)
train_df.loc[:,'spam_score']= train_df['spam_score'].replace(1,-1)
def modify_value(x):
    if x == 0.0 or np.isnan(x):
        return np.nan
    elif x > 0.0:
        return 1
    else:
        return -1
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].apply(modify_value)
train_df.loc[:, 'vadar'] = train_df['vadar'].apply(modify_value)
train_df.rename(columns = {'sentiment_scores':'sentiment_scores2'}, inplace = True)


In [ ]:
## Nulling out the emotion_scores
train_df.loc[train_df['emoji_scores'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['emoji_scores'] == -1, 'vadar'] = np.nan
train_df.loc[train_df['fake_lables'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['fake_lables'] == -1, 'vadar'] = np.nan
train_df.loc[train_df['spam_score'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['spam_score'] == -1, 'vadar'] = np.nan

In [ ]:
num_rows =1127644
# Split the DataFrame into two parts
first_part = train_df.iloc[:num_rows]
second_part = train_df.iloc[num_rows:]
print(len(second_part))

1142838


In [ ]:
first_part_val= first_part.iloc[:100000]
first_part_training = first_part.iloc[100000:]
second_part_val = second_part.iloc[:100000]
second_part_training = second_part.iloc[100000:]

In [ ]:

# Concatenate the second part first, followed by the first part
train_df = pd.concat([second_part_training, first_part_training]).reset_index(drop=True)
train_df.rename(columns={'sentiment_scores':'sentiment_scores2'}, inplace=True)
print(len(train_df))

2070482


In [ ]:
sam_pred = samuels_pred_df['predictions']
num_rows =1127644
# Split the DataFrame into two parts
first_part_sam = sam_pred.iloc[:num_rows]
second_part_sam = sam_pred.iloc[num_rows:]
first_part_val_sam= first_part_sam.iloc[:100000]
first_part_training_sam = first_part_sam.iloc[100000:]
second_part_val_sam = second_part_sam.iloc[:100000]
second_part_training_sam = second_part_sam.iloc[100000:]
# Concatenate the second part first, followed by the first part
sam_pred_train = pd.concat([second_part_training_sam, first_part_training_sam]).reset_index(drop=True)

In [ ]:
train_df['sentiment_scores'] = sam_pred_train

In [ ]:
val_df = pd.concat([second_part_val, first_part_val]).reset_index(drop=True)
val_df['sentiment_scores'] = pd.concat([second_part_val_sam, first_part_val_sam]).reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores2']]
y_val = val_df['label']
X_val = val_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores2']]
# Split the data into training and testing sets
X_train = X
y_train = y

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1]  # Probability scores for the positive class

accuracy_sam = accuracy_score(y_val, y_pred)
precision_sam = precision_score(y_val, y_pred)
recall_sam = recall_score(y_val, y_pred)
precision_sam_neg = precision_score(y_val, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_val, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8073
precision_pos: 0.7911
recall_pos: 0.8352
precision_neg: 0.8254
recall_neg: 0.7795


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores']]
y_val = val_df['label']
X_val = val_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores']]
# Split the data into training and testing sets
X_train = X
y_train = y

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1]  # Probability scores for the positive class

accuracy_sam = accuracy_score(y_val, y_pred)
precision_sam = precision_score(y_val, y_pred)
recall_sam = recall_score(y_val, y_pred)
precision_sam_neg = precision_score(y_val, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_val, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")


accuracy: 0.8581
precision_pos: 0.8488
recall_pos: 0.8713
precision_neg: 0.8678
recall_neg: 0.8448


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores','sentiment_scores2','vadar']]
y_val = val_df['label']
X_val = val_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores','sentiment_scores2','vadar']]
# Split the data into training and testing sets
X_train = X
y_train = y

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1]  # Probability scores for the positive class

accuracy_sam = accuracy_score(y_val, y_pred)
precision_sam = precision_score(y_val, y_pred)
recall_sam = recall_score(y_val, y_pred)
precision_sam_neg = precision_score(y_val, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_val, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")


accuracy: 0.8583
precision_pos: 0.8498
recall_pos: 0.8706
precision_neg: 0.8673
recall_neg: 0.8461
